In [1]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd "drive/MyDrive/Colab Notebooks/medical_assist_academicway"

/content/drive/MyDrive/Colab Notebooks/medical_assist_academicway


In [3]:
!pwd

/content/drive/MyDrive/Colab Notebooks/medical_assist_academicway


In [4]:
# This will show detailed GPU information if available
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Setup

**NOTE: refer https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf for phi-3 mini setup once**

#### Essential Libraries for Task

The cu121 (CUDA 12.1) version works with your CUDA 12.4 because of backward compatibility within major CUDA versions. CUDA 12.1 libraries are generally compatible with CUDA 12.4, as NVIDIA maintains compatibility within major versions.

Regarding which approach is better:

Using `CMAKE_ARGS="-DLLAMA_CUBLAS=on"` might be slightly more beneficial because:

1. It compiles specifically against your exact CUDA version (12.4)
2. You can add additional optimization flags if needed
3. It ensures CUBLAS is explicitly enabled with your specific GPU drivers

The difference in performance is likely to be minor, but if you want the most optimized build for your specific environment, using the CMAKE_ARGS approach would be preferable.

If you're concerned about getting the best performance, you could cancel the current installation and use:

```language=python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on -DLLAMA_CUDA_F16=on" pip install llama-cpp-python
```

This explicitly enables CUBLAS and FP16 support, which can improve performance on T4 GPUs.


In [5]:
# As our cuda version is 12.4 so using whl > cu124 (inorder not to get version mismatch things by using cu118)
# ref: https://pypi.org/project/llama-cpp-python/

!pip install llama-cpp-python --upgrade --force-reinstall --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

# Alternatively use
#!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python


# Use this F16 support one for faster inference
#!CMAKE_ARGS="-DLLAMA_CUBLAS=on -DLLAMA_CUDA_F16=on" pip install llama-cpp-python

#!CMAKE_ARGS="-DLLAMA_CUBLAS=ON" pip install llama-cpp-python --upgrade --force-reinstall

# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 218.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4127106 sha256=d4e3f0cff1b13d429a8c5cb869ec62e899d599c2aa99d617f91e1e889911be88
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8

In [6]:
!pip install huggingface_hub

In [68]:
!pip install langchain langchain_text_splitters pymupdf langchain-community tiktoken sentence-transformers

In [ ]:
!pip install faiss-gpu

#### Imports

In [4]:
import os
import pickle
import time

In [3]:
from huggingface_hub import hf_hub_download, list_repo_files
from llama_cpp import Llama

ModuleNotFoundError: No module named 'llama_cpp'

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader
import re

ModuleNotFoundError: No module named 'langchain_community'

In [69]:
# Import libraries for text chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [ ]:
# Import libraries for embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch
import time

In [ ]:
from typing import List

---

In [8]:
# List available files to get the correct filename
files = list_repo_files("microsoft/Phi-3-mini-4k-instruct-GGUF")
gguf_files = [f for f in files if f.endswith('.gguf')]
print("Available GGUF files:")
for f in gguf_files:
    print(f)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Available GGUF files:
Phi-3-mini-4k-instruct-fp16.gguf
Phi-3-mini-4k-instruct-q4.gguf


We will use q4 one
> Phi-3-mini-4k-instruct-q4.gguf


✅ **`phi-3-mini-4k-instruct-q4.gguf` is much faster**, especially on:

* CPUs (with `llama.cpp`)
* GPUs with limited memory (e.g., T4, RTX 3050, etc.)
* Google Colab Free (RAM or VRAM limited)

---

### ✅ Recommendation

For **RAG and general inference**, use:

> **`phi-3-mini-4k-instruct-q4.gguf`**

It's **fast**, **efficient**, and retains **most of the accuracy** — ideal for local or Colab usage.


**Lets create directory for storing the model (above one)**

In [ ]:
# Create directory for models
!mkdir -p /content/drive/MyDrive/llm_models

In [9]:
llm_models_dir_path = '/content/drive/MyDrive/llm_models'

In [10]:
phi3_mini_local_path = f"{llm_models_dir_path}/Phi-3-mini-4k-instruct-q4.gguf"

In [ ]:

# Download pre-quantized model if it doesn't exist
if not os.path.exists(phi3_mini_local_path):
    print("Downloading pre-quantized model...")
    hf_hub_download(
        repo_id="microsoft/Phi-3-mini-4k-instruct-GGUF",
        filename="Phi-3-mini-4k-instruct-q4.gguf",
        local_dir=llm_models_dir_path,

        # Colab doesn't always support symlinks well esp. when
        # - Writing to mounted paths (e.g. Google Drive)
        # Using False ensures the model files are copied instead of symlinked
        local_dir_use_symlinks=False
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Phi-3-mini-4k-instruct-q4.gguf:   0%|          | 0.00/2.39G [00:00<?, ?B/s]

## Question Answering using LLM

REMEMBER:

the structure `response["choices"][0]["text"]` is consistent across models when using the llama-cpp-python library. This is because llama.cpp follows the OpenAI API response format.

When you call the model with a prompt, it returns a dictionary with this structure:

```
{
  "id": "...",
  "object": "text_completion",
  "created": timestamp,
  "model": "...",
  "choices": [
    {
      "text": "The generated text response",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length" or "stop"
    }
  ],
  "usage": {
    "prompt_tokens": number,
    "completion_tokens": number,
    "total_tokens": number
  }
}
```

So `response["choices"][0]["text"]` will consistently give you the generated text output regardless of which model you're using with llama-cpp-python.


**Phi 3 Mini 4K Info**

Phi-3-mini-4k-instruct has a **total context window** of 4096 tokens. This means the combined length of:
- Your input prompt
- The model's generated response

cannot exceed 4096 tokens.

For example:
- If your prompt is 1000 tokens
- You could generate up to 3096 tokens in response

The `max_tokens` parameter (set to 1024 in our code) limits how many tokens the model will generate in its response, regardless of how much room is left in the context window.

So while the model *could* theoretically generate up to ~4000 tokens (if your prompt is very short), setting `max_tokens=1024` is a practical limit that:
1. Keeps response times reasonable
2. Provides sufficient detail for medical answers
3. Prevents excessively long outputs

You can adjust this value based on your needs, but 1024 is a good starting point.


#### Loading the model

In [11]:
# Function to load the model
def load_model(model_path):
    """
    Load the llm model (quantized) to be used eith llama.cpp
    """
    print(f'checking model at {model_path} ...')
    if not os.path.exists(model_path):
        print(f"Model not found at {model_path} Please check !.")
        return None

    # Load the model with appropriate parameters
    llm = Llama(
        model_path=model_path,
        n_ctx=4096,  # Context window size - use the full 4K that the model supports
        n_gpu_layers=-1,  # Use all GPU layers
        verbose=False
    )

    return llm

In [12]:
# max tokens - 1024 is good starting point // should be sufficient for comprehensive answers
# temperature - A temperature of 0.3-0.5 would be more appropriate for medical question answering
# - for medical applications, leaning toward more deterministic outputs is generally preferred

# Function to define model parameters
def get_model_parameters(temperature=0.3, top_p=0.9, max_tokens=1024):
    """
    Define parameters for model inference
    """
    return {
        "temperature": temperature,
        "top_p": top_p,
        "max_tokens": max_tokens,
        # todo: decide
        # "top_k": 40,
    }

In [13]:
# model path
print('model path: ', phi3_mini_local_path)

model path:  /content/drive/MyDrive/llm_models/Phi-3-mini-4k-instruct-q4.gguf


In [14]:
!ls -l "/content/drive/MyDrive/llm_models"

total 2337140
-rw------- 1 root root 2393231072 May 10 12:57 Phi-3-mini-4k-instruct-q4.gguf


In [15]:
print(os.path.exists(phi3_mini_local_path))

True


In [16]:
# Test the model loading
model = load_model(phi3_mini_local_path)

checking model at /content/drive/MyDrive/llm_models/Phi-3-mini-4k-instruct-q4.gguf ...


**Helper python function**

In [59]:
def response(query, llm, max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    # * ! this method is provided by academic
    model_output = llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

# TODO: make this method to return (answer, response)
def get_response(query, model, params=dict()):
    """
    Get answer to query
    """
    try:

      prompt = 'N/A'

      pt = params['prompt_template']
      if pt:
        prompt = pt(query)
      else:
        # Create the properly formatted prompt
        prompt = create_prompt(query)

      # Get response from model
      response = model(
          prompt,
          # because 512 or 1024 was consuming considerable amount of time so going with what academic takes as default (ie 128)
          max_tokens=params.get("max_tokens", 128),
          temperature=params.get("temperature", 0.2),
          top_p=params.get("top_p", 0.9),
          top_k=params.get("top_k", 50)
          #top_k=params.get("top_k")
      )
      # TODO: comment below print statement (if not needed to trace !...)
      # print('got response', response)
      return response['choices'][0]['text']
    except Exception as e:
      print('Error whilst getting the response', e)
      return 'Error'

def create_prompt(query):
    """
    Creates a properly formatted prompt for the Phi-3-mini-4k-instruct model

    Args:
        query: The user's question or query

    Returns:
        A formatted prompt string that follows the model's expected format
    """
    # Format following Phi-3 chat template
    formatted_prompt = f"""<|user|>
{query}
<|assistant|>"""

    # ! NOTE: not using `pipeline()` or `pipe()`
    # We're using llama.cpp via the llama-cpp-python binding rather than the Hugging Face Transformers library
    # The Transformers pipeline would be useful if we were using the full HF implementation
    # but for our quantized model with llama.cpp, the direct approach we're using is more appropriate

    return formatted_prompt

In [40]:
def display_response(question, answer, verbose=False, response=None):
    """
    Display the question and model response in a clean, formatted way

    Args:
        question: The question asked to the model
        answer: text response to question
        verbose: Whether to show additional details like token counts
        response: The full response object from the model
    """
    # Print with clear formatting
    print("\n" + "="*80)
    print("📋 QUESTION:")
    print("-"*80)
    print(question)
    print("\n" + "🩺 ANSWER:")
    print("-"*80)
    print(answer)
    print("="*80)

    # Optional verbose output with token information
    if verbose and response:
        usage = response.get("usage", {})
        prompt_tokens = usage.get("prompt_tokens", "N/A")
        completion_tokens = usage.get("completion_tokens", "N/A")
        total_tokens = usage.get("total_tokens", "N/A")

        print("\n📊 STATS:")
        print(f"  • Prompt tokens: {prompt_tokens}")
        print(f"  • Completion tokens: {completion_tokens}")
        print(f"  • Total tokens: {total_tokens}")

In [23]:
if model:
    print("Model loaded successfully!")

Model loaded successfully!


In [24]:
# Test questions from the problem statement
questions = [
    "What is the protocol for managing sepsis in a critical care unit?",
    "What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
]

In [25]:
llm = model

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [46]:
response0 = get_response(questions[0], llm)
display_response(questions[0], response0)

got respoonse {'id': 'cmpl-6ea31136-c85e-4ee5-91e1-1634efd91b43', 'object': 'text_completion', 'created': 1746886655, 'model': '/content/drive/MyDrive/llm_models/Phi-3-mini-4k-instruct-q4.gguf', 'choices': [{'text': " The management of sepsis in a critical care unit follows the Surviving Sepsis Campaign (SSC) guidelines, which are periodically updated. The protocol generally includes the following steps:\n\n1. Early recognition and assessment: Identify patients with suspected sepsis, septic shock, or severe sepsis based on clinical signs, symptoms, and laboratory findings.\n\n2. Immediate resuscitation: Initiate aggressive fluid resuscitation with crystalloids, aiming for a 30 mL/kg bolus within the first 3 hours.\n\n3. Antibiotic therapy: Administer broad-spectrum antibiotics within one hour of recognition, and then de-escalate based on culture results and clinical response.\n\n4. Source control: Identify and treat the source of infection, such as draining abscesses, removing infected

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [51]:
i = 1
response1 = get_response(questions[i], llm)
display_response(questions[i], response1)

got respoonse {'id': 'cmpl-439199b1-d299-4347-9701-6533f3d6a890', 'object': 'text_completion', 'created': 1746887080, 'model': '/content/drive/MyDrive/llm_models/Phi-3-mini-4k-instruct-q4.gguf', 'choices': [{'text': ' Appendicitis is an inflammation of the appendix, a small pouch-like organ located in the lower right abdomen. The common symptoms of appendicitis include:\n\n1. Abdominal pain: The pain usually starts around the navel and then moves to the lower right abdomen. The pain tends to worsen over time and may become severe.\n2. Loss of appetite\n3. Nausea and vomiting\n4. Fever\n5. Abdominal bloating\n6. Constipation or diarrhea\n\nAppendicitis', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 38, 'completion_tokens': 128, 'total_tokens': 166}}

📋 QUESTION:
--------------------------------------------------------------------------------
What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical proc

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [52]:
i = 2
response2 = get_response(questions[i], llm)
display_response(questions[i], response2)

got respoonse {'id': 'cmpl-6288221f-0323-458e-8d79-9a07e6aeca28', 'object': 'text_completion', 'created': 1746887208, 'model': '/content/drive/MyDrive/llm_models/Phi-3-mini-4k-instruct-q4.gguf', 'choices': [{'text': ' Sudden patchy hair loss, also known as alopecia areata, can be caused by various factors, including genetics, autoimmune disorders, and stress. Here are some effective treatments and solutions for addressing this condition:\n\n1. Medications:\n   a. Corticosteroids: Injectable or topical corticosteroids can help reduce inflammation and promote hair regrowth.\n   b. Minoxidil: This is a topical solution that can help stimulate hair growth.\n   c. Immunomodulatory agents:', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 44, 'completion_tokens': 128, 'total_tokens': 172}}

📋 QUESTION:
--------------------------------------------------------------------------------
What are the effective treatments or solutions for addressing sudden patc

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [53]:
i = 3
response3 = get_response(questions[i], llm)
display_response(questions[i], response3)

got respoonse {'id': 'cmpl-f7b91398-3fc6-45ec-9c1d-fbe3f66fbd1d', 'object': 'text_completion', 'created': 1746887284, 'model': '/content/drive/MyDrive/llm_models/Phi-3-mini-4k-instruct-q4.gguf', 'choices': [{'text': " I am not able to diagnose or provide specific treatment recommendations. It is crucial to consult with a qualified healthcare professional for an accurate diagnosis and appropriate treatment plan. However, I can provide you with some general information about potential treatments for brain injuries.\n\nTreatment for brain injuries depends on the severity and type of injury, as well as the individual's overall health. Some common approaches to treating brain injuries include:\n\n1. Medical management: This involves monitoring the patient's vital signs, managing pain, and addressing any immediate medical issues related to the injury", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 36, 'completion_tokens': 128, 'total_tokens': 164}}

📋 

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [54]:
i = 4
response4 = get_response(questions[i], llm)
display_response(questions[i], response4)

got respoonse {'id': 'cmpl-284cbf05-074c-4dd5-bc93-5ee1e876e325', 'object': 'text_completion', 'created': 1746887421, 'model': '/content/drive/MyDrive/llm_models/Phi-3-mini-4k-instruct-q4.gguf', 'choices': [{'text': ' If a person has fractured their leg during a hiking trip, it is crucial to take the following precautions and treatment steps:\n\n1. Safety first:\n   - Ensure the injured person is in a safe location, away from any potential hazards.\n   - If possible, help the person to a stable, flat surface.\n\n2. Call for help:\n   - Contact emergency services or a local rescue team to provide professional medical assistance.\n   - If cell phone service is available, call for help immediately.\n\n3. Immobilize the leg', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 42, 'completion_tokens': 128, 'total_tokens': 170}}

📋 QUESTION:
--------------------------------------------------------------------------------
What are the necessary precautions a

## Question Answering using LLM with Prompt Engineering

In [61]:
combinations = [
    # Combination 1: Highly deterministic (factual focus)
    {
        "name": "Highly Deterministic",
        "temperature": 0.1,
        "top_p": 0.5,
        "max_tokens": 150,
        "prompt_template": lambda q: f"<|user|>\nAnswer this medical question with precise, factual information: {q}\n<|assistant|>"
    },

    # Combination 2: Balanced approach
    {
        "name": "Balanced Approach",
        "temperature": 0.4,
        "top_p": 0.8,
        "max_tokens": 160,
        "prompt_template": lambda q: f"<|user|>\nProvide a comprehensive medical answer to this question: {q}\n<|assistant|>"
    },

    # Combination 3: Step-by-step reasoning
    {
        "name": "Step-by-Step Reasoning",
        "temperature": 0.3,
        "top_p": 0.7,
        "max_tokens": 150,
        "prompt_template": lambda q: f"<|user|>\nAnswer this medical question step-by-step with clear reasoning: {q}\n<|assistant|>"
    },

    # Combination 4: Concise summary
    {
        "name": "Concise Summary",
        "temperature": 0.2,
        "top_p": 0.9,
        "max_tokens": 132,
        "prompt_template": lambda q: f"<|user|>\nProvide a brief, concise answer to this medical question: {q}\n<|assistant|>"
    },

    # Combination 5: Medical expert persona
    {
        "name": "Medical Expert Persona",
        "temperature": 0.3,
        "top_p": 0.85,
        "max_tokens": 135,
        "prompt_template": lambda q: f"<|user|>\nAs an experienced medical specialist, answer this question with your expert knowledge: {q}\n<|assistant|>"
    }
]

In [62]:
def test_combinations(question_index):
  query = questions[question_index]
  for e in combinations:
    ans = get_response(query, llm, params=e)
    display_response(query, ans)
    print('---\n')

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [63]:
test_combinations(0)

got respoonse {'id': 'cmpl-44cfbe41-f337-4ff0-be9f-603d6e1f902b', 'object': 'text_completion', 'created': 1746888719, 'model': '/content/drive/MyDrive/llm_models/Phi-3-mini-4k-instruct-q4.gguf', 'choices': [{'text': ' The protocol for managing sepsis in a critical care unit is based on the Surviving Sepsis Campaign (SSC) guidelines, which emphasize early recognition, prompt administration of antibiotics, and aggressive fluid resuscitation. The following steps are typically followed:\n\n1. Early recognition: Identify patients with suspected sepsis by assessing for signs and symptoms, such as fever, elevated heart rate, altered mental status, and hypotension.\n\n2. Immediate interventions:\n   a. Administer broad-spectrum antibiotics within one hour of recognition.\n   b. Initiate fluid resuscitation with', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 31, 'completion_tokens': 150, 'total_tokens': 181}}

📋 QUESTION:
--------------------------------

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [64]:
test_combinations(1)

got respoonse {'id': 'cmpl-5029667d-19d0-42b1-9168-99689acf86d1', 'object': 'text_completion', 'created': 1746889097, 'model': '/content/drive/MyDrive/llm_models/Phi-3-mini-4k-instruct-q4.gguf', 'choices': [{'text': ' Appendicitis is an inflammation of the appendix, a small, finger-like pouch that projects from the large intestine. The common symptoms of appendicitis include:\n\n1. Abdominal pain: The pain usually starts around the navel and then moves to the lower right side of the abdomen. The pain typically worsens over time and becomes more severe.\n\n2. Loss of appetite\n\n3. Nausea and vomiting\n\n4. Low-grade fever\n\n5. Constipation or diarrhea\n\n6. Abdominal bloating\n\n7. Inability to pass gas\n\n8. Abdominal', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 49, 'completion_tokens': 150, 'total_tokens': 199}}

📋 QUESTION:
--------------------------------------------------------------------------------
What are the common symptoms of appe

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [65]:
test_combinations(2)

got respoonse {'id': 'cmpl-ddf847b5-d0b0-458c-aea1-cc3e3692feeb', 'object': 'text_completion', 'created': 1746889448, 'model': '/content/drive/MyDrive/llm_models/Phi-3-mini-4k-instruct-q4.gguf', 'choices': [{'text': " Sudden patchy hair loss, also known as alopecia areata, is an autoimmune condition where the body's immune system mistakenly attacks hair follicles, leading to localized bald spots on the scalp. The exact cause of alopecia areata is unknown, but it is believed to involve a combination of genetic and environmental factors.\n\nPossible causes of alopecia areata include:\n\n1. Genetic predisposition: A family history of alopecia areata or other autoimmune diseases may increase the risk of developing the condition.\n2. Immune system dysfunction: An overactive immune system may attack hair follicles, causing", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 55, 'completion_tokens': 150, 'total_tokens': 205}}

📋 QUESTION:
------------------

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [66]:
test_combinations(3)

got respoonse {'id': 'cmpl-656cabf2-698f-4380-a037-2bcf4436b795', 'object': 'text_completion', 'created': 1746889776, 'model': '/content/drive/MyDrive/llm_models/Phi-3-mini-4k-instruct-q4.gguf', 'choices': [{'text': " Treatment for a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function, depends on the severity and location of the injury, as well as the specific symptoms and complications experienced by the individual. Here are some general treatment options:\n\n1. Immediate medical attention: In the case of a severe head injury, immediate medical attention is crucial. This may involve stabilizing the patient's vital signs, performing a thorough neurological examination, and obtaining imaging studies such as a CT scan or MRI to assess the extent of the injury.\n\n2. Medications: Various medications may be prescribed to manage symptoms and complications associated with", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'pro

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [67]:
test_combinations(4)

got respoonse {'id': 'cmpl-89eecbf0-a8dc-49d0-861e-892e738e0ec0', 'object': 'text_completion', 'created': 1746890563, 'model': '/content/drive/MyDrive/llm_models/Phi-3-mini-4k-instruct-q4.gguf', 'choices': [{'text': ' Precautions and treatment steps for a person who has fractured their leg during a hiking trip include:\n\n1. Immobilization: Immobilize the injured leg using a splint or a makeshift support to prevent further injury.\n\n2. Pain management: Administer over-the-counter pain medications, such as acetaminophen or ibuprofen, to alleviate pain and reduce inflammation.\n\n3. Elevation: Elevate the injured leg above heart level to reduce swelling and improve blood circulation.\n\n4. Ice application: Apply ice packs wrapped in a cloth to the injured area for 15-', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 53, 'completion_tokens': 150, 'total_tokens': 203}}

📋 QUESTION:
---------------------------------------------------------------------

### 👀 **Summary**



We utilized the Phi-3 Mini 4K model in a GPU-accelerated Colab setup to generate medical responses for a predefined set of five clinically relevant questions.

itial tests showed that a max_tokens setting of 512 to 1024 consumes too much time So we resort back to defualt provided by academia (ie 128)

1. **Response Time vs. Token Length**: Generating 512 tokens took significantly longer than shorter outputs. There appears to be a near-linear relationship between max_tokens and generation time.

2. **Temperature Impact**: Lower temperature settings (0.1-0.2) produced more consistent, factual responses appropriate for medical information, while slightly higher values (0.3-0.4) introduced minor variations in phrasing without compromising accuracy.

3. **Prompt Engineering Effects**: Directive prompts (e.g., "Answer step-by-step") noticeably influenced the structure of responses, with the model generally following the requested format.

4. **Default Value Optimization**: We settled on max_tokens=128 as the default for a balance between response quality and generation speed. This value provided sufficient detail for most medical questions while maintaining reasonable response times.

5. **Optimal Balance**: The "Balanced Approach" (temp=0.3, top_p=0.8) provided a good compromise between factual accuracy and natural language flow for medical questions.

6. **Conciseness vs. Completeness**: While the "Concise Summary" setting generated faster responses, some medical questions benefited from the additional context provided by longer outputs.

7. **Persona Framing**: The "Medical Expert Persona" prompt appeared to elicit slightly more technical terminology and structured explanations compared to neutral prompts.

8. **Token Efficiency**: Lower temperatures generally resulted in more information-dense responses, requiring fewer tokens to convey key medical information.

9. **Model Limitations**: For complex medical protocols (like sepsis management), even the longest responses (384 tokens) sometimes felt truncated before completing the full explanation.

***The responses generated by Phi-3 Mini were surprisingly coherent and informative, even without retrieval augmentation — suggesting that smaller models can still be leveraged for meaningful domain-specific reasoning when guided with the right prompt and sampling parameters.***

## Data Preparation for RAG

**Prep**

In [72]:
!pwd

/content/drive/MyDrive/Colab Notebooks/medical_assist_academicway


In [70]:
!ls -la

total 19851
-rw------- 1 root root    46430 May 10 08:48 medical_assistant_learner_notebook.ipynb
-rw------- 1 root root 20150490 May  9 11:14 medical_diagnosis_manual.pdf
-rw------- 1 root root   125661 May 10 15:39 notebook1.ipynb
drwx------ 2 root root     4096 May 10 11:56 phi3_models


In [71]:
pdf_path = 'medical_diagnosis_manual.pdf'

In [84]:
# helper Functions

def load_pdf_with_langchain(pdf_path):
    """
    Load a PDF file using LangChain's PyMuPDFLoader

    Args:
        pdf_path: Path to the PDF file

    Returns:
        List of document objects with page content and metadata
    """
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"PDF file not found at {pdf_path}")

    # Load the PDF using PyMuPDFLoader
    loader = PyMuPDFLoader(pdf_path)
    documents = loader.load()

    print(f"PDF loaded successfully: {os.path.basename(pdf_path)}")
    print(f"Total pages: {len(documents)}")

    return documents

def preview_documents(documents, num_pages=2):
    """
    Preview the first few pages of loaded documents
    """
    total_pages = len(documents)

    print(f"Previewing first {min(num_pages, total_pages)} pages:")
    print("="*80)

    for i in range(min(num_pages, total_pages)):
        doc = documents[i]
        text = doc.page_content

        print(f"\n--- Page {i+1} ---\n")
        print(text[:1000] + "..." if len(text) > 1000 else text)

    print("\n" + "="*80)

### Loading the Data

In [82]:
try:
  documents = load_pdf_with_langchain(pdf_path)
except:
  print(f"Error loading pdf: {e}")

PDF loaded successfully: medical_diagnosis_manual.pdf
Total pages: 4114


#### Checking the first 5 pages

In [85]:
# preview the PDF
preview_documents(documents, 5)

Previewing first 5 pages:

--- Page 1 ---

nipunshah6776@gmail.com
0W3XG8QC4A
nt for personal use by nipunshah6776@
shing the contents in part or full is liable

--- Page 2 ---

nipunshah6776@gmail.com
0W3XG8QC4A
This file is meant for personal use by nipunshah6776@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

--- Page 3 ---

Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...........................................................................................................................................................

#### Checking the number of pages

In [87]:
print('Total number of pages: ', len(documents))

Total number of pages:  4114


### Data Overview

In [86]:
 # Combine all text for statistics
full_text = "\n\n".join([doc.page_content for doc in documents])

# Display some statistics about the content
lines = full_text.split('\n')
words = re.findall(r'\w+', full_text)

print(f"\nPDF loaded successfully with {len(documents)} pages and {len(full_text)} characters")
print(f"Approximate number of lines: {len(lines)}")
print(f"Approximate number of words: {len(words)}")


PDF loaded successfully with 4114 pages and 13703595 characters
Approximate number of lines: 207112
Approximate number of words: 2027501


--- FROM HERE RUN ALL CELLS (AS NOT RUN YETP

### Data Chunking

Using BAAI/bge-small-en-v1.5 (not an OpenAI model), the standard `RecursiveCharacterTextSplitter` would be more appropriate than the tiktoken version.

Here's why:

1. `RecursiveCharacterTextSplitter.from_tiktoken_encoder()` uses OpenAI's tokenization, which doesn't align with how BGE tokenizes text

2. The tokenization mismatch could lead to suboptimal chunk boundaries for your embedding model

3. The standard `RecursiveCharacterTextSplitter` with character-based chunking provides more consistent results across different embedding models

4. For BGE models, character-based chunking with appropriate chunk size and overlap settings works well in practice


This approach will be more aligned with your chosen embedding model and provide more consistent results.


In [ ]:
def create_chunks_from_documents(documents, chunk_size=512, chunk_overlap=50):
    """
    Create chunks from documents using character-based splitting

    Args:
        documents: List of documents from PyMuPDFLoader
        chunk_size: Size of chunks in characters
        chunk_overlap: Number of characters to overlap between chunks

    Returns:
        List of chunked documents
    """
    # Create a character-based splitter (better for non-OpenAI embedding models like BGE)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )

    # Split the documents into chunks
    chunks = text_splitter.split_documents(documents)

    print(f"Created {len(chunks)} chunks from {len(documents)} pages")
    print(f"Average chunk length: {sum(len(doc.page_content) for doc in chunks) / len(chunks):.2f} characters")

    return chunks

In [ ]:
def preview_chunks(chunks, num_chunks=2):
    """
    Preview a sample of the created chunks

    Args:
        chunks: List of document chunks
        num_chunks: Number of chunks to preview
    """
    print(f"\nPreviewing {min(num_chunks, len(chunks))} chunks out of {len(chunks)}:")

    for i in range(min(num_chunks, len(chunks))):
        chunk = chunks[i]
        print(f"\n--- Chunk {i+1} ---")

        # # Print metadata if available
        # if hasattr(chunk, 'metadata') and chunk.metadata:
        #     print(f"Source: {chunk.metadata.get('source')}, Page: {chunk.metadata.get('page')}")

        # Print content preview
        content = chunk.page_content
        print(f"Length: {len(content)} characters")
        print(f"Content preview: {content[:150]}...")

In [ ]:
chunks = create_chunks_from_documents(documents, chunk_size=512, chunk_overlap=50)

In [ ]:
preview_chunks(chunks)

### Embedding

In [ ]:
# ? For medical text retrieval specifically, BAAI/bge-small-en-v1.5 offers the best balance of speed and retrieval
EMBEDDING_MODEL_NAME = "BAAI/bge-small-en-v1.5"
STORE_PATH = "faiss_index"

In [ ]:
def create_embedding_model(model_name, device="cuda"):
    """
    Create an embedding model using the specified model

    Args:
        model_name: Name of the HuggingFace model to use
        device: Device to run the model on ("cuda" or "cpu")

    Returns:
        Configured embedding model
    """
    # Check if CUDA is available when device is set to "cuda"
    if device == "cuda" and not torch.cuda.is_available():
        print("CUDA not available, falling back to CPU")
        device = "cpu"

    # Configure the embedding model
    model_kwargs = {'device': device}
    encode_kwargs = {'normalize_embeddings': True}  # Normalize for cosine similarity

    # Create the embedding model
    embedding_model = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

    print(f"Embedding model created: {model_name} on {device}")
    return embedding_model

### Vector Database

In [ ]:
def load_or_create_faiss_vectorstore(
    chunks,
    embedding_model,
    persist_directory=STORE_PATH,
):
    """
    Create a FAISS vector store from chunks or load from disk if it exists

    Args:
        chunks: List of document chunks
        embedding_model: The embedding model to use
        persist_directory: Directory to save/load the FAISS index

    Returns:
        FAISS vector store
    """
    index_path = os.path.join(persist_dir, "index.faiss")
    metadata_path = os.path.join(persist_dir, "index.pkl")

    # Load if already exists
    if os.path.exists(index_path) and os.path.exists(metadata_path):
        print(f"Loading FAISS index from {persist_dir}")

        # Load the existing index
        vector_store = FAISS.load_local(persist_directory, embedding_model)
        print(f"Loaded FAISS index with {vector_store.index.ntotal} vectors")

        return vector_store

    # Create a new vector store if it doesn't exist
    print("Creating new FAISS index from documents...")
    start_time = time.time()

    # Create the vector store from documents
    vector_store = FAISS.from_documents(chunks, embedding_model)

    # Else create from documents
    print(f"Creating new FAISS index at {persist_dir}")
    vectorstore = FAISS.from_documents(documents, embedding_model)

    elapsed_time = time.time() - start_time
    print(f"FAISS index created in {elapsed_time:.2f} seconds")
    print(f"Index contains {vector_store.index.ntotal} vectors of dimension {vector_store.index.d}")

    # Create the directory if it doesn't exist
    os.makedirs(persist_directory, exist_ok=True)

    # Save the index to disk
    print(f"Saving FAISS index to {persist_directory}")
    vector_store.save_local(persist_directory)

    vectorstore.save_local(persist_dir)

    return vectorstore

In [ ]:
def print_faiss_store_info(store):
    print(f"Total documents: {len(store.docstore._dict)}")
    print(f"Vector dimension: {store.index.d}")
    print(f"Index type: {type(store.index).__name__}")

### Retriever

In [ ]:
def retrieve_from_faiss(vector_store, query, top_k=3):
    """
    Retrieve the top_k most similar chunks for a given query

    Args:
        vector_store: FAISS vector store
        query: Query string
        k: Number of chunks to retrieve (default: 5)

    Returns:
        List of retrieved documents
    """
    # Retrieve the top_k most similar documents and their scores
    docs_and_scores = vector_store.similarity_search_with_score(query, k=top_k)

    # # Separate documents and scores
    # docs = [doc for doc, score in docs_and_scores]
    # scores = [score for doc, score in docs_and_scores]

    print(f"Retrieved {len(docs_and_scores)} documents for query: '{query}'")

    docs = []

    for i, (doc, score) in enumerate(docs_and_scores):
        docs.append(doc)

        print(f"Rank {i+1}:")
        print(f"Score: {score:.4f}")

        # # Print metadata if available
        # if hasattr(doc, 'metadata') and doc.metadata:
        #     source = doc.metadata.get('source', 'Unknown')
        #     page = doc.metadata.get('page', 'Unknown')
        #     print(f"Source: {source}, Page: {page}")

        # Print content preview (documents are objects with page_content attribute)
        content = doc.page_content
        preview_length = min(200, len(content))
        print(f"Content: {content[:preview_length]}...\n")

    return docs

### System and User Prompt Template

In [ ]:
RAG_DEFAULT_SYSTEM_MSG = """You are a helpful medical assistant. Your task is to answer medical questions based solely on the provided context from the Merck Manual.

Guidelines:
- Answer ONLY based on the context provided.
- If the context doesn't contain the answer, say "I don't have enough information to answer this question".
- Be concise and accurate in your responses.
- For medical conditions, include key information about symptoms, diagnosis, and treatment if available.
- Do not fabricate information or use knowledge outside the provided context."""
# As for now we are not including metadata in user_message putting this below point aside !!
# - If relevant and possible, specify which section of the Merck Manual the information comes from."""


In [ ]:
def create_user_message(query, retrieved_docs, inclue_metadata=FALSE):
    """
    Create the user message portion of the RAG prompt in Phi-3-mini format

    Format
    ```
    Context:
    {context}

    Question:
    ```

    Args:
        query: User's question
        retrieved_docs: List of retrieved documents
        num_docs: Number of documents to include

    Returns:
        (Formatted user message string with context first, then question,
        context)
    """
    # Start with the instruction
    user_message = "Use the following context to answer the question.\n\nContext:"

    context_value = ""

    # Add the most relevant documents to the context
    for i, doc in enumerate(retrieved_docs):
        # Add document content
        context_value += f"\n\nDocument {i+1}:"

        # Add metadata if available
        if inclue_metadata and hasattr(doc, 'metadata') and doc.metadata:
            source = doc.metadata.get('source', '')
            page = doc.metadata.get('page', '')
            if source or page:
                context_value += f" (Source: {source}, Page: {page})"

        # Add content
        context_value += f"\n{doc.page_content}"

    user_message += context_value

    # Add the question after the context
    user_message += f"\n\nQuestion:\n{query}"

    return user_message, context_value

In [ ]:
def create_phi3_rag_prompt(system_message, user_message):
    """
    Create a RAG prompt specifically formatted for Phi-3-mini

    Format:-
    ```
    <|system|>
    You are a helpful and concise assistant. Answer based only on the provided context. If the context does not contain enough information, say "I don't know."

    <|user|>
    Context:
    {context}

    Question:
    {question}
    <|assistant|>
    ```

    Args:
        query: User's question
        retrieved_docs: List of retrieved documents
        num_docs: Number of documents to include in the prompt

    Returns:
        Formatted prompt string ready for Phi-3-mini
    """
    # Phi-3-mini uses a specific format with <|system|>, <|user|>, and <|assistant|> tags
    # Combine into Phi-3 format
    # phi3_prompt = f"<|system|>\n{system_message}\n<|user|>\n{user_message}\n<|assistant|>"
    # ..
    phi3_prompt = (
        f"<|system|>\n"
        f"{system_message}\n\n"
        f"<|user|>\n"
        f"{user_message}\n\n"
        f"<|assistant|>"
    )

    return phi3_prompt

In [ ]:
def get_rag_response(query, vector_store, llm_model, llm_model_params=dict(), context_k=3, system_message=RAG_DEFAULT_SYSTEM_MSG):
    """
    End-to-end RAG pipeline to answer medical questions

    Args:
        query: User's medical question
        vector_store: FAISS vector store containing Merck Manual chunks
        llm_model: The Phi-3-mini model object
        llm_model_params: params such as temperature, top_k, .. dictonary to pass to llm model during inferencing
        context_k: Number of relevant chunks to retrieve

    Returns:
        Generated (answer, context) from the LLM
    """
    print(f"Processing question: '{query}'")

    # Step 1: Retrieve relevant documents
    print("\n--- Retrieving relevant documents ---")
    retrieved_docs = retrieve_from_faiss(vector_store, query, top_k=context_k)

    # Step 2: Create user message
    print("\n--- Creating user message ---")
    user_message, context_val = create_user_message(query, retrieved_docs)

    # Step 3: Create RAG prompt
    print("\n--- Creating user message ---")
    phi3_prompt = create_phi3_rag_prompt(system_message, user_message)

    # Step 4: Generate answer using existing LLM function
    print("\n--- Generating answer ---")
    answer = get_response(phi3_prompt, llm_model, params=llm_model_params)

    print("\n--- Answer ---")
    print(f"{answer[:125]}...") # show only first 125 characters

    return (answer, context_val)

### Configuration & Usage

In [ ]:
# Configure embedding model
embedding_model = create_embedding_model()
vector_store = create_or_load_vector_store(chunks, embedding_model)

In [ ]:
# Show some meta info about store
print_faiss_store_info(vector_store)

## Question Answering using RAG

**Helper class that helps to hold rag tasks info** !!

In [ ]:
# Helper class

class RAGMeta:
    def __init__(self, num_questions):
        # Initialize the list with a fixed size of `num_questions`, all set to None initially
        self.meta = [None] * num_questions

    def register(self, question_num, question, answer, context):
        # Ensure the question_num is within bounds
        if 0 <= question_num < len(self.meta):
            # Register the information for the specified question number
            self.meta[question_num] = (question, answer, context)
        else:
            raise IndexError("Question number is out of range")

    def get(self, question_num):
        # Retrieve information for the given question number
        if 0 <= question_num < len(self.meta):
            return self.meta[question_num]
        else:
            return None  # If question_num doesn't exist yet

    def get_all(self):
        return list(self.meta)

In [ ]:
# For medical RAG applications with the Merck Manual, a `top_k` value of 3 - 5 is optimal
TO_FETCH_DOC = 3 # from store

In [ ]:
rag_meta = RAGMeta(num_questions=5)

In [ ]:
def process_question_for_rag(i):
  print(f'Process Question {i} initiated ---\n')
  (ans, ctxt) = get_rag_response(questions[i], vector_store, llm, context_k=TO_FETCH_DOC)
  print('\n ---- View ---')
  display_response(questions[i], response1)

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
i = 0
(que, ans, ctxt) = process_question_for_rag(i)
rag_meta.register(i, que, ans, ctxt)

### **Fine-tuning**

In [ ]:
# RAG parameter combinations for testing
rag_parameter_combinations = [
    {
        "top_p": 0.7,
        "context_k": 3,
        "temperature": 0.1,
        "desc": "Conservative sampling with focused context and low temperature for maximum factual accuracy"
    },
    {
        "top_p": 0.9,
        "context_k": 3,
        "temperature": 0.2,
        "desc": "Wider token sampling but focused context with low temperature for balanced precision"
    },
    {
        "top_p": 0.7,
        "context_k": 5,
        "temperature": 0.3,
        "desc": "Conservative sampling with focused context but higher temperature for more natural responses"
    },
    {
        "top_p": 0.8,
        "context_k": 5,
        "temperature": 0.1,
        "desc": "Wider token sampling with expanded context but low temperature for comprehensive yet precise answers"
    },
    {
        "top_p": 0.8,
        "context_k": 3,
        "temperature": 0.3,
        "desc": "Wider token sampling with expanded context and moderate temperature for detailed and varied responses"
    }
]

In [ ]:
def test_rag_combinations(question_index):
  query = questions[question_index]
  for e in combinations:
    (ans, _) = get_rag_response(query, vector_store, llm, llm_model_params=e, context_k=e['context_k'])
    display_response(query, ans)
    print('---\n')

## Output Evaluation

In [ ]:
# helper function

def build_rag_eval_prompt(context: str, question: str, answer: str) -> str:
    system_message = """You are an expert and impartial evaluator of AI-generated answers for medical questions.

Your task is to assess the quality of an answer to a given question based on the provided context.

Evaluate the answer along two dimensions:

1. Groundedness:
   - Is the answer supported solely by the provided context?
   - Does it avoid including information not present in the context?

2. Relevance:
   - Does the answer address the question using appropriate information from the context?
   - Is the answer complete, relevant, and clearly written?

Scoring guidelines:
- Groundedness:
  1 = Contains significant information not in the context or contradicts the context
  2 = Contains some information not supported by the context
  3 = Mostly grounded with minor additions not in the context
  4 = Fully grounded with very minor phrasing not explicitly in the context
  5 = Perfectly grounded, all information comes directly from the context

- Relevance:
  1 = Does not address the question at all
  2 = Partially addresses the question but misses key aspects
  3 = Addresses the main question but lacks some important details
  4 = Addresses the question well with most relevant details
  5 = Perfectly addresses the question with all relevant information from the context

Return your evaluation in the following JSON format:

{
  "groundedness_assessment": "<brief justification>",
  "groundedness_score": <score from 1 to 5>,
  "relevance_assessment": "<brief justification>",
  "relevance_score": <score from 1 to 5>
}

Keep your assessment justifications brief (1-2 sentences each).
Be objective and concise in your justifications. For medical information, accuracy is particularly important.
"""

    return f"""<|system|>
{system_message}

<|user|>
Context:
{context}

Question:
{question}

Answer:
{answer}

Evaluate the answer based on the above criteria and return the output in the specified JSON format.

<|assistant|>"""

In [ ]:
def display_evaluation_results(question, answer, evaluation_result):
    """
    Display the evaluation results for a given question and answer.

    Parameters:
    - question (str): The question asked.
    - answer (str): The AI-generated answer.
    - evaluation_result (dict): The evaluation result containing 'groundedness_assessment', 'groundedness_score',
      'relevance_assessment', and 'relevance_score'.
    """
    print("\nEvaluation Results for Question: ", question)
    print("="*50)

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

    print("\nEvaluation:")
    print("-" * 50)
    print(f"Groundedness Assessment: {evaluation_result['groundedness_assessment']}")
    print(f"Groundedness Score: {evaluation_result['groundedness_score']}")
    print(f"Relevance Assessment: {evaluation_result['relevance_assessment']}")
    print(f"Relevance Score: {evaluation_result['relevance_score']}")
    print("="*50)

In [ ]:
evaluation_model_settings = {
    "max_tokens": 256,  # Sufficient for concise evaluation with brief justifications
    "temperature": 0,   # Ensures deterministic, objective output. Avoids randomness.
    "top_p": 0.7        # Use full probability space, but with temperature=0.0, it's moot
}

In [ ]:
eval_results = []

for i, (question, answer, context) in enumerate(rag_meta):
    eval_prompt = build_eval_prompt(context=context, question=question, answer=answer)
    evaluation = get_response(eval_prompt, llm, params=evaluation_model_settings)  # your existing inference function
    eval_results.append((i, evaluation))
    display_evaluation_results(question, answer, evaluation)

## Actionable Insights and Business Recommendations